In [1]:
from tqdm import tqdm
import pandas as pd
import regex

In [2]:
with open('deribit_BTC-PERPETUAL_20230130_depth10.txt', 'r') as f:
    data = f.read()
data_splited = data.split('\n')
data_splited_ = data_splited[2:-2]

In [3]:
total_records = len(data_splited_)
i = 0
pbar = tqdm(total=total_records)
OrderbookSnapshots_records = []
Trades_records = []

  0%|                                                                                                                           | 0/219572735 [00:00<?, ?it/s]

In [4]:
while i < total_records:
    dct = {}
    record = data_splited_[i]
    type = record.split('type: ')[1].split(' symbol')[0]
    
    if type == 'OrderbookSnapshot':
        dct['receive_time'] = record.split('receive_time: ')[1].split(' server_time')[0]
        dct['server_time'] = record.split('server_time: ')[1].split(' orderbook')[0]
        dct['ask_size'] = int(record.split('ask_size: ')[1])
        dct['bid_size'] = int(record.split('bid_size: ')[1].split(' ask_size')[0])
        
        
        ################ PROCESS ASKS #########################
        dct['asks_price'] = []
        dct['asks_volume'] = []
        for _ in range(min(dct['ask_size'], 10)):
            i += 1
            pbar.update(1)
            
            record = data_splited_[i]
            dct['asks_price'].append(float(record.split('Ask price: ')[1].split(' volume')[0]))
            dct['asks_volume'].append(float(record.split('volume: ')[1]))
        
        if min(dct['ask_size'], 10) != 10:
            dct['asks_price'].extend([-1]*(10 - dct['ask_size']))
            dct['asks_volume'].extend([-1]*(10 - dct['ask_size']))
            
        ################ PROCESS BIDS #########################
        dct['bids_price'] = []
        dct['bids_volume'] = []
        for _ in range(min(dct['bid_size'], 10)):
            i += 1
            pbar.update(1)
            
            record = data_splited_[i]
            dct['bids_price'].append(float(record.split('Bid price: ')[1].split(' volume')[0]))
            dct['bids_volume'].append(float(record.split('volume: ')[1]))
            
        if min(dct['bid_size'], 10) != 10:
            dct['bids_price'].extend([-1]*(10 - dct['bid_size']))
            dct['bids_volume'].extend([-1]*(10 - dct['bid_size']))
        
        OrderbookSnapshots_records.append(dct)
        
        i += 2
        pbar.update(2)
    elif type == 'Trades':
        
        dct['receive_time'] = record.split('receive_time: ')[1].split(' server_time')[0]
        dct['server_time'] = record.split('server_time: ')[1].split(' trades')[0]
        trades = data_splited_[i].split('trades: ')[1]
        if len(regex.findall('Bid', trades)) == 1:
            dct['type'] = 'Bid'
        elif len(regex.findall('Ask', trades)) == 1:
            dct['type'] = 'Ask'
        dct['price'] = float(trades.split()[3])
        dct['volume'] = float(trades.split()[4])
        
        Trades_records.append(dct)
        
        i += 1
        pbar.update(1)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 219572735/219572735 [11:50<00:00, 507131.79it/s]

In [5]:
df_dct = {'receive_time': [x['receive_time'] for x in OrderbookSnapshots_records],
              'server_time': [x['server_time'] for x in OrderbookSnapshots_records],
              'ask_size': [x['ask_size'] for x in OrderbookSnapshots_records],
              'bid_size': [x['bid_size'] for x in OrderbookSnapshots_records]}

for i in range(10):
    df_dct[f'ask_price_{i}']  = [x['asks_price'][i] for x in OrderbookSnapshots_records]
    df_dct[f'ask_volume_{i}'] = [x['asks_volume'][i] for x in OrderbookSnapshots_records]
for i in range(10):
    df_dct[f'bid_price_{i}']  = [x['bids_price'][i] for x in OrderbookSnapshots_records]
    df_dct[f'bid_volume_{i}'] = [x['bids_volume'][i] for x in OrderbookSnapshots_records]
    
orderbook_df = pd.DataFrame(df_dct)

In [6]:
df_dct = {'receive_time': [x['receive_time'] for x in Trades_records],
          'server_time': [x['server_time'] for x in Trades_records],
          'type': [x['type'] for x in Trades_records],
          'price': [x['price'] for x in Trades_records],
          'volume': [x['volume'] for x in Trades_records]}
    
trades_df = pd.DataFrame(df_dct)

In [9]:
orderbook_df.to_parquet('deribit_ob.parquet')

In [10]:
trades_df.to_parquet('deribit_trades.parquet')